In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

from bs4 import BeautifulSoup
import re
import requests
import string

In [30]:
client = dataiku.api_client()
source_data_project = client.get_project('SOURCEPROJECTGUTENBERGDATA')

BASE_URL = "https://www.gutenberg.org"

resp = requests.get(BASE_URL + "/ebooks/bookshelf/136")
soup = BeautifulSoup(resp.text, "lxml")

next_page = True

while next_page:
    next_page_attr = soup.find("li", attrs={"class": "statusline"})
    next_page_link = next_page_attr.find_all("a")
    
    link = ""
    for l in next_page_link:
        if l.text == "Next":
            link = l["href"]

    book_links = soup.find_all("li", attrs={"class": "booklink"})

    for book in book_links:
        book_id = re.findall(r'\d+', book.find("a")["href"])[0]
        book_title = book.find("span", attrs={"class": "title"}).text
        
        book_title_cleaned = book_title \
            .translate(str.maketrans('', '', string.punctuation)) \
            .lower() \
            .replace("'", "") \
            .replace(" ", "_")
            
        print(book_title_cleaned)
        
        try:
            params = {
                'customConfig': {
                    'book_id': book_id,
                    'mirror': 'http://aleph.gutenberg.org'
                },
                'notReadyIfEmpty': False,
                'previewAllPartitions': False
            }

            ds = source_data_project.create_dataset(dataset_name=book_title_cleaned, params=params, type='CustomPython_gutenberg-project-book')
            ds.set_schema({'columns': [{'name': 'id', 'type': 'string'},
               {'name': 'author', 'type': 'string'},
               {'name': 'title', 'type': 'string'},
               {'name': 'text', 'type': 'string'}],
              'userModified': True}
            )
            print("completed: {}, {}".format(book_title_cleaned, book_id))
        except Exception as e:
            print("{} encountered error: {}".format(book_title_cleaned, e))
        
    if link:
        resp = requests.get(BASE_URL + link)
        soup = BeautifulSoup(resp.text, "lxml")
    else:
        next_page = False

the_federalist_papers
completed: the_federalist_papers, 1404
the_frontier_in_american_history
completed: the_frontier_in_american_history, 22994
history_of_the_united_states
completed: history_of_the_united_states, 16960
the_oregon_trail_sketches_of_prairie_and_rockymountain_life
completed: the_oregon_trail_sketches_of_prairie_and_rockymountain_life, 1015
the_declaration_of_independence_of_the_united_states_of_america
completed: the_declaration_of_independence_of_the_united_states_of_america, 16780
comic_history_of_the_united_states
completed: comic_history_of_the_united_states, 21427
the_shirley_letters_from_california_mines_in_185152
completed: the_shirley_letters_from_california_mines_in_185152, 23280
the_united_states_constitution
completed: the_united_states_constitution, 5
the_federalist_papers
the_federalist_papers encountered error: java.lang.IllegalArgumentException: Dataset SOURCEPROJECTGUTENBERGDATA.the_federalist_papers already exists
captains_of_the_civil_war_a_chronicle_o

completed: abraham_lincoln_and_the_union_a_chronicle_of_the_embattled_north, 2836
a_school_history_of_the_united_states
completed: a_school_history_of_the_united_states, 11313
the_old_merchant_marine_a_chronicle_of_american_ships_and_sailors
completed: the_old_merchant_marine_a_chronicle_of_american_ships_and_sailors, 3099
amendments_to_the_united_states_constitution
completed: amendments_to_the_united_states_constitution, 19582
the_negro_at_work_in_new_york_city_a_study_in_economic_progress
completed: the_negro_at_work_in_new_york_city_a_study_in_economic_progress, 24712
the_day_of_the_confederacy_a_chronicle_of_the_embattled_south
completed: the_day_of_the_confederacy_a_chronicle_of_the_embattled_south, 3035
the_sequel_of_appomattox_a_chronicle_of_the_reunion_of_the_states
completed: the_sequel_of_appomattox_a_chronicle_of_the_reunion_of_the_states, 2897
the_constitution_of_the_united_states
completed: the_constitution_of_the_united_states, 10065
a_report_of_the_debates_and_proceedin